In [1]:
%%capture
%cd ..

In [ ]:
test_time:str = 'Test_Time(s)'
date_time:str = 'Date_Time'
step_time:str = 'Step_Time(s)'
c_index:str = 'Cycle_Index'
step_index:str = 'Step_Index'
current:str = 'Current(A)'
voltage:str = 'Voltage(V)'
charge_cap:str = 'Charge_Capacity(Ah)'
discharge_cap:str = 'Discharge_Capacity(Ah)'
channel:str = 'Channel'
dataset_root = 'src/dataset'
ds_cleaned = f'{dataset_root}/cleaned'
index = 'Index'

charge_indexes = [
    1,
    3,
    5,
    6,
    8,
    7,
    9,
]
discharge_indexes = [
    1,
    2,
    3,
    4,
    5,
    6,
    8,
    9,
]
white_list = [
    test_time,
    date_time,
    step_time,
    c_index,
    step_index,
    voltage,
    current,
    charge_cap,
    discharge_cap]

charge_list = [
    step_index,
    c_index,
    current,
    voltage,
    charge_cap
]

discharge_list = [
    step_index,
    c_index,
    current,
    voltage,
    discharge_cap
]

In [5]:
import pandas as pd
import os
from utils import constants as cs
from datetime import datetime
import os


# merge all files in specific folder
def merge_files (data_dir: str = 'dataset/CS2_38', file_name:str = 'CS2_38', is_charge = True):
    # define new data frame
    cleaned = pd.DataFrame()
    
    # consider only channel sheets
    for filename in os.listdir(data_dir):
        xls_file = pd.ExcelFile(os.path.join(data_dir, filename))
        sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith(cs.channel)]
        # cycle all files 
        for sheet_fname in sheet_fnames:
            df = pd.read_excel(os.path.join(data_dir, filename), sheet_name=sheet_fname)
            df = df[cs.white_list]
            # convert datetime
            df[cs.date_time] = df[cs.date_time].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").timestamp())
            # find negative rows
            # df = df[(df[cs.test_time] < 0) | (df[cs.step_time] < 0) | (df[cs.date_time] < 0)]
            df = df[~((df[cs.test_time] < 0) | (df[cs.step_time] < 0) | (df[cs.date_time] < 0))]

            # drop invalid rows
            # df = df[~(invalid_rows)]
            # find max cycle value for each sheet
            max_cindex = cleaned[cs.c_index].max() if not df.empty else 0
            # update total cycle index
            df[cs.c_index] += max_cindex
            # concat all files 
            cleaned = pd.concat([cleaned, df], ignore_index=True)
            if is_charge: 
                # consider only step index 2-4
                cleaned = cleaned[~cleaned[cs.step_index].isin(cs.charge_indexes)]
                # check current
                cond_sindex = (cleaned[cs.step_index] == 4)
                cond_curr = (cleaned[cs.current] > 0.55)
                # reduce dataframe
                cleaned = cleaned[~(cond_sindex & cond_curr)]
                for key, group in cleaned.groupby(cs.c_index):
                    diffs =  group[cs.current].diff().abs() 
                    mask = ~(diffs > 0.0502)
                    cleaned.loc[cleaned[cs.c_index] == key, :] = cleaned.loc[cleaned[cs.c_index] == key, :][mask]
                cleaned = cleaned[cs.charge_list]
                cleaned.to_csv(os.path.join(cs.ds_cleaned, f'charge-{file_name}'), index=False)    
            else:
                # consider only step index 7
                cleaned = cleaned[~cleaned[cs.step_index].isin(cs.discharge_indexes)]
                cleaned = cleaned[cs.discharge_list]
                cleaned.to_csv(os.path.join(cs.ds_cleaned, f'discharge-{file_name}'), index=False)    


In [6]:
ds_root = os.listdir(cs.dataset_root)

for folder in ds_root:
    if folder.startswith('CS2'):
        merge_files(data_dir = f'{cs.dataset_root}/{folder}', file_name = f'{folder}.csv')   
        merge_files(data_dir = f'{cs.dataset_root}/{folder}', file_name = f'{folder}.csv', is_charge=False)

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'src/dataset'

In [ ]:
# Imports
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from utils.constants import white_list

In [ ]:
# Some constants values
data_dir: str = "dataset/CS2_38"
cleaned = "dataset/cleaned"
df = pd.DataFrame()
hi = pd.DataFrame()

current = 'Current(A)'
voltage = 'Voltage(V)'
c_index = 'Cycle_Index'
# time = 'Test_Time(s)'
t0_voltage = 3.8
t1_voltage = 4.1999

datasets = {}

In [ ]:
CS2_38_cleaned = pd.DataFrame()

for filename in os.listdir(data_dir):
    xls_file = pd.ExcelFile(os.path.join(data_dir, filename))
    sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]

    # print(filename)
    for sheet_fname in sheet_fnames:
        df_temp = pd.read_excel(os.path.join(data_dir, filename), sheet_name=sheet_fname)
        df_temp = df_temp[white_list]
        # Trovare il massimo valore di Cycle_Index in CS2_38_cleaned
        max_combined_cycle_index = CS2_38_cleaned['Cycle_Index'].max() if not CS2_38_cleaned.empty else 0
        print(max_combined_cycle_index)
        # Aggiornare i valori di Cycle_Index nel DataFrame corrente
        df_temp['Cycle_Index'] += max_combined_cycle_index
        print(df_temp['Cycle_Index'])
        # Aggiungere il DataFrame corrente a CS2_38_cleaned
        CS2_38_cleaned = pd.concat([CS2_38_cleaned, df_temp], ignore_index=True)

In [ ]:
CS2_38_cleaned.to_csv(os.path.join(cleaned, 'CS2_38_cleaned.csv'), index=False)

In [ ]:
path: str = 'dataset/cleaned/CS2_38_cleaned.csv'
# Carica il DataFrame dal tuo file CSV
df_concat = pd.read_csv(path)
df_concat['Index'] = df_concat.index
max_voltage = df_concat[voltage].max()
min_voltage = df_concat[voltage].min()

# mean_voltage = df_concat[voltage].mean()
# mean_voltage = (mean_voltage - min_voltage) / (max_voltage - min_voltage)
# std_voltage = df_concat[voltage].std()
# std_voltage = (std_voltage - min_voltage) / (max_voltage - min_voltage)


In [ ]:
df_concat

In [ ]:
print(max_voltage, min_voltage)

In [ ]:

for key, group in df_concat.groupby(c_index):
    t0_index = (group[voltage] >= t0_voltage).idxmax()
    t1_index = (group[voltage] >= t1_voltage).idxmax()
    group[voltage] = (group[voltage] - min_voltage) / (max_voltage - min_voltage)
    
    mean_voltage = group[voltage].mean()
    std_voltage = group[voltage].std()
    group[voltage] = (group[voltage]- mean_voltage) / std_voltage
    selected_rows = group[(group['Index'] >= t0_index) & (group['Index'] <= t1_index)]
    hi_v = selected_rows[voltage].sum()
    # hi_v = np.trapz(selected_rows[voltage])
    cycle = group[c_index].iloc[0]
    # for i in range(t0_index, t1_index):
    #     hi_v = 
    # hi_v = group[voltage].iloc[t0_index:t1_index + 1].sum()

    print(cycle)
    # group[voltage].to_csv(f'dataset/cycles/{key}.csv')
    row = pd.DataFrame([{'hi_v': hi_v, 'cycle': cycle}])
    hi = pd.concat([hi, row])

In [ ]:
print(hi)

In [ ]:
hi.to_csv(f'{cleaned}/hi.csv')


In [ ]:
plt.plot( hi["cycle"], hi["hi_v"])
plt.show()


In [ ]:
for filename in os.listdir(data_dir):
    xls_file = pd.ExcelFile(f'{data_dir}/{filename}')
    sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]
    for sheet_fname in sheet_fnames:
        df_temp = pd.read_excel(f'{data_dir}/{filename}', sheet_name=sheet_fname)
        df_temp = df_temp[white_list]
        max_voltage = df_temp[voltage].max()
        min_voltage = df_temp[voltage].min()
        
        # df_temp = df_temp[(df_temp[voltage] >= min_voltage) & (df_temp[voltage] <= max_voltage)]
        df_temp = df_temp[df_temp[voltage].diff().gt(0)]
        
        
        # mean_voltage = df_temp[voltage].mean()
        # std_voltage = df_temp[voltage].std()

        #Standardization
        # df_temp[voltage] = (df_temp[voltage] - mean_voltage) / std_voltage

        

        grouped_df = df_temp.groupby(df_temp[c_index])

        for key, group in grouped_df:
            t0_index = (group[voltage] >= t0_voltage).idxmax()
            t1_index = (group[voltage] >= t1_voltage).idxmax()
            #Normalization
            group[voltage] = (group[voltage] - min_voltage) / (max_voltage - min_voltage)
            # print( group[voltage] )
            # voltage_normalized = (group[voltage] - min_voltage) / (max_voltage - min_voltage)
            # hi_v = np.trapz(group[voltage])
            # hi_v = group[voltage].sum()
            hi_v = np.trapz(group[voltage].iloc[t0_index:t1_index + 1])
            # hi_v = group[voltage].iloc[t0_index:t1_index + 1].sum()
            # hi_v = 0
            # for i in range(t0_index, t1_index-1):
            #     hi_v += group[voltage].iloc[i]
            row = pd.DataFrame([{'hi_v': hi_v}])
            hi = pd.concat([hi, row])
            print("ale puzza:",max_voltage, min_voltage)


In [ ]:
print(hi)

In [ ]:
plt.plot(hi_v)

In [ ]:
hi.to_csv(f'{cleaned}/hi.csv')

In [ ]:
# #Cycle all excel files in the same folder
# for filename in os.listdir(data_dir):
#     xls_file = pd.ExcelFile(f'{data_dir}/{filename}')
#     sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]
    
#     df_list = []  # List to accumulate DataFrames for each sheet
    
#     for sheet_fname in sheet_fnames:
#         df_temp = pd.read_excel(f'{data_dir}/{filename}', sheet_name=sheet_fname)
#         df_temp = df_temp[[voltage, current, c_index  ]]
#         df_temp = df_temp[(df_temp[voltage] >= min_voltage) & (df_temp[voltage] <= max_voltage)]
#         df_temp = df_temp[df_temp[voltage].diff().gt(0)]
        
#         df_list.append(df_temp)  # Append the DataFrame to the list
    
#     datasets[filename] = pd.concat(df_list).to_numpy()

# # Now, concatenate the values from all datasets
# df_cycle = np.concatenate([dataset[:, 0] for dataset in datasets.values()])
# df_values = np.concatenate([np.delete(dataset, 0, axis=1) for dataset in datasets.values()])

# # Calculate mean and std
# X_mean, X_std = df_values.mean(0), df_values.std(0)

# # Normalize values
# df_values = (df_values - X_mean) / X_std

# # Min and max after normalization
# X_min, X_max = df_values.min(), df_values.max()